### Universidad Nacional de Lujan - Bases de Datos Masivas (11088) - Cavasin Nicolas #143501
# TP05-01 - Árboles de decisión

### Ejercicio 3:
Ahora, analice el archivo *zoo.csv*. Genere el árbol de decisión que permita inferir el tipo de animal en función de sus características. Explique someramente que resultado se obtiene en términos del árbol y en términos de la eficiencia del mismo.
- ¿Varía ese resultado si se elimina el atributo “animal”?¿Por qué?

- Cuantos niveles posee el árbol generado? ¿Qué atributos debemos modificar si deseamos realizar una poda del mismo? Modifique esos atributos para que el árbol generado conste de 4 niveles. ¿Afecta la eficiencia de la clasificación esta modificación?
    

In [1]:
!rm zoo.csv
!wget https://raw.githubusercontent.com/bdm-unlu/2020/master/TPs/TP05/TP0501/zoo.csv


--2020-11-09 20:23:23--  https://raw.githubusercontent.com/bdm-unlu/2020/master/TPs/TP05/TP0501/zoo.csv
Loaded CA certificate &#39;/etc/ssl/certs/ca-certificates.crt&#39;
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.216.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.216.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10171 (9.9K) [text/plain]
Saving to: &#39;zoo.csv&#39;

zoo.csv             100%[===================&gt;]   9.93K  --.-KB/s    in 0.006s  

2020-11-09 20:23:24 (1.68 MB/s) - &#39;zoo.csv&#39; saved [10171/10171]



In [2]:
# Importo pandas para manipular los datos
import pandas as pd

# Leo el archivo
zoo = pd.read_csv('zoo.csv')

# Muestro algunos datos
zoo.head()

,animal,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,type
0,aardvark,True,False,False,True,False,False,True,True,True,True,False,False,4,False,False,True,mammal
1,antelope,True,False,False,True,False,False,False,True,True,True,False,False,4,True,False,True,mammal
2,bass,False,False,True,False,False,True,True,True,True,False,False,True,0,True,False,False,fish
3,bear,True,False,False,True,False,False,True,True,True,True,False,False,4,False,False,True,mammal
4,boar,True,False,False,True,False,False,True,True,True,True,False,False,4,True,False,True,mammal


In [3]:
print(f'Cantidad de tuplas: {zoo.shape[0]}')
print(f'Cantidad de columnas: {zoo.shape[1]}')

Cantidad de tuplas: 101
Cantidad de columnas: 18


Como se puede observar, los datos pertenecientes a las columnas *animal* y *type* son del tipo *String*. Los árboles de clasificación no aceptan features del tipo categórico (solo numéricas), pero sí targets.  

Por lo tanto, se deberán separar features y targets para luego aplicar la transformación correspondiente a la columna *animal*.

In [4]:
# Importo la libreria para aplicar la transformación
from sklearn import preprocessing

# Instancio el codificador
encoder = preprocessing.LabelEncoder()

# Defino el target y lo elimino del dataset
target_name = 'type'
target = zoo.pop(target_name)

# Creo una lista con los nombres de las features para almacenarlos
features_names = []

# Recorro cada columna del dataframe
for col in zoo.columns:

    # Agrego cada nombre al features_names
    features_names.append(col)

    # Si col es del tipo string lo transformo a numerico
    if zoo[col].dtype == object:
        zoo[col] = encoder.fit_transform(zoo[col])

# Obtengo features
features = zoo[features_names]

# Acomoda el encoder de acuerdo a los diferentes valores del target
encoder.fit(target)

# Obtengo los nombres de dichos valores del target
target_classes = encoder.classes_

print('Nombre de las features:', *features_names, end='\n\n')
print(f'Nombre del target: {target_name}')

# Muestro como quedaron los datos transformados
zoo.head()

Nombre de las features: animal hair feathers eggs milk airborne aquatic predator toothed backbone breathes venomous fins legs tail domestic catsize

Nombre del target: type


,animal,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize
0,0,True,False,False,True,False,False,True,True,True,True,False,False,4,False,False,True
1,1,True,False,False,True,False,False,False,True,True,True,False,False,4,True,False,True
2,2,False,False,True,False,False,True,True,True,True,False,False,True,0,True,False,False
3,3,True,False,False,True,False,False,True,True,True,True,False,False,4,False,False,True
4,4,True,False,False,True,False,False,True,True,True,True,False,False,4,True,False,True


Se aplicó una transformación por el método *LabelEncoder* a la columna *animal* asignándo un número a cada valor *String* encontrado en la columna procesada.  

In [5]:
# Importo la libreria que permite separar en training y test set
from sklearn.model_selection import train_test_split as s

# Importo el arbol
from sklearn import tree

# Importo libreria para graficar
import graphviz

# Importo las metricas para testear el modelo
from sklearn import metrics

# Divido el dataset en 70-30 para training y test respectivamente
features_train, features_test, target_train, target_test = s(features, target, random_state=0, test_size=0.3)

# Instancio el arbol por entropia
t = tree.DecisionTreeClassifier(criterion='entropy')

# Lo entreno
t.fit(features_train, target_train)

# Almaceno el modelo en formato DOT
zoo_dot = tree.export_graphviz(t
                    , out_file=None
                    , feature_names=features_names
                    , class_names=target_classes
                    , label='all'
                    , filled=True, rounded=True
                    , special_characters=True)

# Tambien lo guardo como PNG
graph = graphviz.Source(zoo_dot)
graph.format = 'png'
graph.render('zoo')

# Obtengo la prediccion con el set de prueba definido anteriormente
prediccion = t.predict(features_test)

# Convierto a array para poder imprimirlo + facil
target_test = pd.array(target_test)

# Muestro prediccion vs original
header = '{:<4}{:<2}{:<20}{:<5}{:<20}'.format('#', '|', 'Predicción', '|', 'Original')
print(header)
print('='*50, end='')
sep = '|'
for i in range(len(prediccion)):
    print(f"\n{str(i).ljust(4, ' ')}{sep.ljust(2, ' ')}{prediccion[i].ljust(20, ' ')}{sep.ljust(5, ' ')}{target_test[i].ljust(20, ' ')}\n", end='-'*50)

# Veo qué tan acertado estuvo
print(f'\n\nPrecisión del modelo: {metrics.accuracy_score(target_test, prediccion)}', end='\n\n')

# Muestro un reporte de clasificación con diferentes métricas sobre cada feature 
print(f'Reporte de clasificación: \n{metrics.classification_report(target_test, prediccion)}')

#   | Predicción          |    Original            
0   | amphibian           |    amphibian           
--------------------------------------------------
1   | fish                |    fish                
--------------------------------------------------
2   | fish                |    fish                
--------------------------------------------------
3   | mammal              |    mammal              
--------------------------------------------------
4   | mammal              |    mammal              
--------------------------------------------------
5   | mammal              |    mammal              
--------------------------------------------------
6   | bird                |    bird                
--------------------------------------------------
7   | fish                |    fish                
--------------------------------------------------
8   | mammal              |    mammal              
--------------------------------------------------
9   | mammal         

Árbol de clasificación sobre el dataset en estado original:

![Arbol](zoo.png)

***Observaciones:***
- El modelo posee una profundidad de 6 niveles.
- La hoja con menor cantidad de samples posee 1 y la de mayor cantidad posee 30 samples, de todos animales mamíferos.
- El modelo obtenido se encuentra muy desbalanceado hacia izquierda. 
- El sub-árbol que comienza en el nivel 1 está balanceado.
- Aparentemente el dataset posee varios animales no-mamíferos (reptiles, insectos, aves, etc.) y la feature de mayor GI es "¿Se alimenta el animal con leche?" que se corresponde con la columna *milk*. 
- Al dividir utilizando dicha feature se obtiene un gran desbalanceo en la construcción del árbol ante la dificultad para clasificar todos los registros de animales no-mamíferos.
- A pesar de lo indicado acerca del desbalanceo, el modelo parecería clasificar bien porque las métricas indican una gran precisión del 96.7%, reportando un único error: confusión de *fish* con *anfibio*.


___


A continuación se eliminará la columna *animal* para observar el comportamiento del modelo resultante.

In [6]:
# Elimino animal de las features
features.pop('animal')
features_names.remove('animal')

# Nuevamente divido el dataset en 70-30 para training y test respectivamente
features_train, features_test, target_train, target_test = s(features, target, random_state=0, test_size=0.3)

# Instancio el arbol por entropia
t2 = tree.DecisionTreeClassifier(criterion='entropy')

# Lo entreno
t2.fit(features_train, target_train)

# Almaceno el modelo en formato DOT
zoo_dot = tree.export_graphviz(t2
                    , out_file=None
                    , feature_names=features_names
                    , class_names=target_classes
                    , label='all'
                    , filled=True, rounded=True
                    , special_characters=True)

# Tambien lo guardo como PNG
graph = graphviz.Source(zoo_dot)
graph.format = 'png'
graph.render('zoo_sin_animal')

# Obtengo la prediccion con el set de prueba definido anteriormente
prediccion2 = t2.predict(features_test)

# Convierto a array para poder imprimirlo + facil
target_test = pd.array(target_test)

# Veo qué tan acertado estuvo
print(f'\n\nPrecisión del modelo: {metrics.accuracy_score(target_test, prediccion2)}', end='\n\n')

# Muestro un reporte de clasificación con diferentes métricas sobre cada feature 
print(f'Reporte de clasificación: \n{metrics.classification_report(target_test, prediccion2)}')



Precisión del modelo: 0.967741935483871

Reporte de clasificación: 
              precision    recall  f1-score   support

   amphibian       0.50      1.00      0.67         1
        bird       1.00      1.00      1.00         6
        fish       1.00      1.00      1.00         7
      insect       1.00      1.00      1.00         2
invertebrate       1.00      1.00      1.00         2
      mammal       1.00      1.00      1.00        11
     reptile       1.00      0.50      0.67         2

    accuracy                           0.97        31
   macro avg       0.93      0.93      0.90        31
weighted avg       0.98      0.97      0.97        31



Árbol de clasificación **sin** la columna *animal*.

![Arbol](zoo_sin_animal.png)

***Observaciones:***
- El árbol resultante se ve afectado estructuralmente en el ante-último sub-árbol de la izquierda. Es decir, el hijo izquierdo del nodo que se corresponde con la columna *predator*.
- A nivel rendimiento, la precisión se mantuvo en 96.7% con un único error de clasificación: *fish* por *reptile*.
- Esto se debe a que el único valor que aporta la columna *animal* es el nombre de cada animal del dataset, ergo su valor de Ganancia de Información es muy bajo para la clasificación por Entropía y por eso se ubica en el nivel 4 (anteúltimo) del modelo original.
- Debido al uso del parámetro ``random_state=0`` en la separación de training y testing, los datos se desordenan para evitar repeticiones. Por lo tanto, a pesar de que la tasa de precisión se mantenga los errores de clasificación pueden variar porque los  sets de training y testing cambiaron y no por algún bug.

___


A continuación se realizarán dos podas diferentes para que al árbol alcance 4 niveles de profundidad:
- Primero se usará el parámetro ``max_depth=4`` para limitar la profundidad.
- Luego ``min_samples_leaf=5`` para limitar la cantidad mínima de samples por hoja (esto causa que el árbol se retraiga).

In [7]:
# Nuevamente divido el dataset en 70-30 para training y test respectivamente
features_train, features_test, target_train, target_test = s(features, target, random_state=0, test_size=0.3)

# Instancio el arbol por entropia y limito a 4 niveles
t3 = tree.DecisionTreeClassifier(criterion='entropy', max_depth=4)

# Lo entreno
t3.fit(features_train, target_train)

# Almaceno el modelo en formato DOT
zoo_dot = tree.export_graphviz(t3
                    , out_file=None
                    , feature_names=features_names
                    , class_names=target_classes
                    , label='all'
                    , filled=True, rounded=True
                    , special_characters=True)

# Tambien lo guardo como PNG
graph = graphviz.Source(zoo_dot)
graph.format = 'png'
graph.render('zoo_limite_niveles')

# Obtengo la prediccion con el set de prueba definido anteriormente
prediccion3 = t3.predict(features_test)

# Convierto a array para poder imprimirlo + facil
target_test = pd.array(target_test)

# Veo qué tan acertado estuvo
print(f'\n\nPrecisión del modelo: {metrics.accuracy_score(target_test, prediccion3)}', end='\n\n')

# Muestro un reporte de clasificación con diferentes métricas sobre cada feature 
print(f'Reporte de clasificación: \n{metrics.classification_report(target_test, prediccion3)}')



Precisión del modelo: 0.9032258064516129

Reporte de clasificación: 
              precision    recall  f1-score   support

   amphibian       0.50      1.00      0.67         1
        bird       1.00      1.00      1.00         6
        fish       0.88      1.00      0.93         7
      insect       1.00      0.50      0.67         2
invertebrate       0.67      1.00      0.80         2
      mammal       1.00      1.00      1.00        11
     reptile       0.00      0.00      0.00         2

    accuracy                           0.90        31
   macro avg       0.72      0.79      0.72        31
weighted avg       0.87      0.90      0.88        31



Árbol de clasificación con un *máximo* de 4 niveles de profundidad:

![Arbol](zoo_limite_niveles.png)

In [8]:
# Nuevamente divido el dataset en 70-30 para training y test respectivamente
features_train, features_test, target_train, target_test = s(features, target, random_state=0, test_size=0.3)

# Instancio el arbol por entropia y limito a 4 niveles
t4 = tree.DecisionTreeClassifier(criterion='entropy', min_samples_leaf=5)

# Lo entreno
t4.fit(features_train, target_train)

# Almaceno el modelo en formato DOT
zoo_dot = tree.export_graphviz(t4
                    , out_file=None
                    , feature_names=features_names
                    , class_names=target_classes
                    , label='all'
                    , filled=True, rounded=True
                    , special_characters=True)

# Tambien lo guardo como PNG
graph = graphviz.Source(zoo_dot)
graph.format = 'png'
graph.render('zoo_limite_samples')

# Obtengo la prediccion con el set de prueba definido anteriormente
prediccion4 = t4.predict(features_test)

# Convierto a array para poder imprimirlo + facil
target_test = pd.array(target_test)

# Veo qué tan acertado estuvo
print(f'\n\nPrecisión del modelo: {metrics.accuracy_score(target_test, prediccion4)}', end='\n\n')

# Muestro un reporte de clasificación con diferentes métricas sobre cada feature 
print(f'Reporte de clasificación: \n{metrics.classification_report(target_test, prediccion4)}')



Precisión del modelo: 0.9032258064516129

Reporte de clasificación: 
              precision    recall  f1-score   support

   amphibian       0.50      1.00      0.67         1
        bird       1.00      1.00      1.00         6
        fish       0.88      1.00      0.93         7
      insect       1.00      0.50      0.67         2
invertebrate       0.67      1.00      0.80         2
      mammal       1.00      1.00      1.00        11
     reptile       0.00      0.00      0.00         2

    accuracy                           0.90        31
   macro avg       0.72      0.79      0.72        31
weighted avg       0.87      0.90      0.88        31



Árbol de clasificación con *límite mínimo* de 5 samples por hoja (y por ende con 4 niveles de profundidad):

![Arbol](zoo_limite_samples.png)

## Conclusiones:
- Ambas podas poseen la misma precisión de 90.32%, sin embargo son estructuralmente diferentes.

- La poda que impone un *límite máximo* de profundidad utilizando el parámetro ``max_depth`` posee un sub-árbol izquierdo más que la poda que impone un *límite mínimo* de 5 samples por hoja.

- El rendimiento de ambas variantes **disminuyó** un ~6% con respecto al modelo original y además no permite identificar animales del tipo *reptile* por la falta de profundidad.